# Práctica 2: Derivación e integración numérica

Autor: José Alberto Hoces Castro

3º Doble Grado en Ingeniería Informática y Matemáticas

## Ejercicios

In [1]:
# Importamos los módulos que van a ser necesarios para la realización de los ejercicios

import numpy as np # Importamos el módulo NumPy con el pseudónimo np
import sympy as sp # Importamos el módulo SymPy con el pseudónimo sp
# sp.init_printing() # con esta opción obtendremos bonitas fórmulas en la pantalla
# pero en las últimas versiones de Python parece ser que no hace falta indicarlo

# Aquí importaremos dentro del módulo de gráficos Matplotlib el submódulo
import matplotlib.pyplot as plt  #  Pyplot con el pseudónimo plt
# usaremos una opción para poder visualizar los gráficos incrustados     
#  en línea dentro de este mismo notebook    
%matplotlib inline  

from random import random 
# Importamos la función generadora de números pseudoaleatorios

def mychop(expr, *, max=10**(-15)): 
    if abs(expr) > max:
      return expr 
    else:
      return 0

1.- Obtenga mediante interpolación en el espacio $\mathbb{P}_2$ una fórmula para aproximar $f''(a)$ del tipo combinación
de $f(a-h)$, $f(a)$ y $f(a+h)$.

In [2]:
f = sp.Function('f')
a,h = sp.symbols('a,h')
x = [a-h,a,a+h] # Estos serán los nodos de interpolación

In [3]:
y = [f(x[0]),f(x[1]),f(x[2])]  # Estos serán los valores interpolados

In [4]:
z = sp.Symbol('z')  # Utilizaremos z como variable simbólica
p0 = y[0]  # Comenzamos a construir el polinomio de interpolación
p1 = p0 + (z-x[0])/(x[1]-x[0])*(y[1]-y[0]) # usando la idea de Newton

In [5]:
D = sp.Symbol('D')   # D será la correspondiente diferencia dividida
p2 = p1 + (z-x[0])*(z-x[1])*D # D = f[x0,x1,x2]
p2  # ya tenemos la expresión genérica de dicho polinomio de grado 2

D*(-a + z)*(-a + h + z) + f(a - h) + (f(a) - f(a - h))*(-a + h + z)/h

In [6]:
p2.subs({z:x[0]})==y[0],p2.subs({z:x[1]})==y[1] 
# Comprobamos que interpola y0 e y1

(True, True)

In [7]:
# Pero también se debe garantizar que p(x2) = y2
sol2=sp.solve(p2.subs({z:x[2]})-y[2],D)  # para ello resolvemos
sol2    # La ecuación correspondiente y despejamos el valor de D

[(-2*f(a) + f(a - h) + f(a + h))/(2*h**2)]

In [8]:
D = sol2[0]
p2 = p1 + (z-x[0])*(z-x[1])*D
p2  # Obtenemos el polinomio final de interpolación

f(a - h) + (f(a) - f(a - h))*(-a + h + z)/h + (-a + z)*(-a + h + z)*(-2*f(a) + f(a - h) + f(a + h))/(2*h**2)

In [9]:
p2.subs({z:x[2]})==y[2] # Comprobamos que también interpola el valor y[2]

True

Ahora para obtener la correspondiente fórmula de derivación numérica, bastará con derivar dicho polinomio respecto de la variable independiente dos veces y evaluar en el punto donde queremos aproximar la segunda derivada.

In [10]:
sp.diff(p2,z,2)

(-2*f(a) + f(a - h) + f(a + h))/h**2

2.- Con la fórmula obtenida en el ejercicio 1, halle una tabla de aproximaciones y errores de $f_1''(2.5)$, siendo $f_1(x)=x^x$, para $h=10^{-i},\; i=1,\ldots,5.$

In [11]:
def formula(f,a,h):
    return (-2*f(a)+f(a-h)+f(a+h))/(h**2)

In [12]:
def f1(z):
    return z**z

In [13]:
# Guardamos el valor de la segunda derivada de f1 en 2.5 para el posterior
# cálculo de los errores

valor_exacto = sp.diff(f1(z),z,2).subs({z:2.5})
valor_exacto

40.2416648563875

In [14]:
# Calculamos las 5 iteraciones que nos piden

aproximaciones = []
for i in range(1,6):
    aproximaciones.append(formula(lambda z: z**z, 2.5, 10**(-i)))

aproximaciones

[40.42056829795832,
 40.243450230939004,
 40.24168270788664,
 40.24166475602442,
 40.24164113047845]

In [15]:
# Calculamos los 5 errores asociados a las aproximaciones

errores = []
for i in range(5):
    errores.append(abs(valor_exacto - aproximaciones[i]))

errores

[0.178903441570853,
 0.00178537455153815,
 1.78514991731049e-5,
 1.00363045874019e-7,
 2.37259090170028e-5]

3.- Sea $f_2(x)=\frac{x^2+40}{x+\sqrt{5x}+7}$. Calcule una tabla que recoja las derivadas de $f_2$ en $x_i=1,2,\ldots,10$, utilizando alguna de las fórmulas de derivación numérica de primer orden obtenidas al inicio de la práctica, con $h=10^{-3}$, y muestre al mismo tiempo el error cometido en cada punto. Repita el ejercicio con la fórmula centrada obtenida para la derivada primera y, finalmente, para la obtenida en el ejercicio 1 (con respecto a la segunda derivada).

In [16]:
def f2(z):
    return (z**2 + 40)/(z + (5*z)**(1/2) + 7)

Para calcular las derivadas de primer orden en $x_i=1,2,\ldots,10$ comenzaremos usando la primera fórmula de todas que se deduce en los apuntes de la práctica:

$f'(a) \approx p'(a) = \frac{f(a+h)-f(a)}{h}$

In [17]:
def formula1(f,a,h):
    return (-f(a)+f(a+h))/h

In [18]:
valores_exactos = []
for i in range(1,11):
    valores_exactos.append(sp.diff(f2(z),z).subs({z:i}))

valores_exactos

[-0.633413841504903,
 -0.203729991363422,
 0.0135536765957583,
 0.152356382446352,
 0.250865051903114,
 0.325234486346073,
 0.383753089267232,
 0.431201820656649,
 0.470566739057635,
 0.503824070415537]

In [19]:
aproximaciones = []
for i in range(1,11):
    aproximaciones.append(formula1(f2, i, 10**(-3)))

aproximaciones

[-0.6330758508230616,
 -0.20358841102519065,
 0.013637834543889227,
 0.15241382963759875,
 0.2509073591920874,
 0.32526720196468517,
 0.3837792735330581,
 0.43122332479583747,
 0.47058475905004116,
 0.5038394181333672]

In [20]:
errores = []
for i in range(10):
    errores.append(abs(valores_exactos[i] - aproximaciones[i]))

In [21]:
# Finalmente mostramos en formato de tabla las aproximaciones y sus errores

print("FÓRMULA USADA: {}\n".format(formula1(f,a,h)))
print("x_i\tValor exacto\t\tAproximación\t\tError")
print("-\t------------\t\t------------\t\t-----")

for i in range(10):
    print("{}\t{}\t{}\t{}".format(i+1, valores_exactos[i], aproximaciones[i], errores[i]))

FÓRMULA USADA: (-f(a) + f(a + h))/h

x_i	Valor exacto		Aproximación		Error
-	------------		------------		-----
1	-0.633413841504903	-0.6330758508230616	0.000337990681841038
2	-0.203729991363422	-0.20358841102519065	0.000141580338231473
3	0.0135536765957583	0.013637834543889227	0.0000841579481308807
4	0.152356382446352	0.15241382963759875	0.0000574471912467844
5	0.250865051903114	0.2509073591920874	0.0000423072889731979
6	0.325234486346073	0.32526720196468517	0.0000327156186125110
7	0.383753089267232	0.3837792735330581	0.0000261842658260680
8	0.431201820656649	0.43122332479583747	0.0000215041391883886
9	0.470566739057635	0.47058475905004116	0.0000180199924058044
10	0.503824070415537	0.5038394181333672	0.0000153477178297390


Ahora vamos a hacer exactamente lo mismo pero para la fórmula centrada para la derivada primera:

$f'(a) \approx p'(a) = \frac{f(a+h)-f(a-h)}{2h}$

In [22]:
def formula2(f,a,h):
    return (-f(a-h)+f(a+h))/(2*h)

In [23]:
aproximaciones = []
for i in range(1,11):
    aproximaciones.append(formula2(f2, i, 10**(-3)))

aproximaciones

[-0.6334139834538455,
 -0.20373002121565342,
 0.013553664382381925,
 0.15235637597976748,
 0.25086504797688924,
 0.3252344837485488,
 0.38375308744642567,
 0.4312018193228795,
 0.4705667380475731,
 0.50382406963001]

In [24]:
errores = []
for i in range(10):
    errores.append(abs(valores_exactos[i] - aproximaciones[i]))

In [25]:
# Finalmente mostramos en formato de tabla las aproximaciones y sus errores

print("FÓRMULA USADA: {}\n".format(formula2(f,a,h)))
print("x_i\tValor exacto\t\tAproximación\t\tError")
print("-\t------------\t\t------------\t\t-----")

for i in range(10):
    print("{}\t{}\t{}\t{}".format(i+1, valores_exactos[i], aproximaciones[i], errores[i]))

FÓRMULA USADA: (-f(a - h) + f(a + h))/(2*h)

x_i	Valor exacto		Aproximación		Error
-	------------		------------		-----
1	-0.633413841504903	-0.6334139834538455	1.41948942822268E-7
2	-0.203729991363422	-0.20373002121565342	2.98522312980332E-8
3	0.0135536765957583	0.013553664382381925	1.22133764213217E-8
4	0.152356382446352	0.15235637597976748	6.46658449010573E-9
5	0.250865051903114	0.25086504797688924	3.92622495626327E-9
6	0.325234486346073	0.3252344837485488	2.59752386266854E-9
7	0.383753089267232	0.38375308744642567	1.82080633903681E-9
8	0.431201820656649	0.4312018193228795	1.33376959476905E-9
9	0.470566739057635	0.4705667380475731	1.01006225605715E-9
10	0.503824070415537	0.50382406963001	7.85527420887888E-10


Y por último volvemos a realizar de nuevo 10 aproximaciones con sus respectivos errores pero de la segunda derivada con la fórmula deducida en el ejercicio 1:

$f''(a) \approx p'(a) = \frac{-2f(a) + f(a-h) + f(a+h)}{h^2}$

In [26]:
valores_exactos = []
for i in range(1,11):
    valores_exactos.append(sp.diff(f2(z),z,2).subs({z:i}))

valores_exactos

[0.676265098285376,
 0.283220364176106,
 0.168340319928121,
 0.114907312895053,
 0.0846224302869937,
 0.0654364313639429,
 0.0523721743690358,
 0.0430109449028751,
 0.0360420057237485,
 0.0306970066620211]

In [27]:
aproximaciones = []
for i in range(1,11):
    aproximaciones.append(formula(f2, i, 10**(-3)))

aproximaciones

[0.6762652615677212,
 0.2832203809255418,
 0.1683403230146041,
 0.11490731566254908,
 0.08462243039630835,
 0.06543643227274742,
 0.05237217326481414,
 0.04301094591596666,
 0.036042004936120975,
 0.03069700671431974]

In [28]:
errores = []
for i in range(10):
    errores.append(abs(valores_exactos[i] - aproximaciones[i]))

In [29]:
# Finalmente mostramos en formato de tabla las aproximaciones y sus errores

print("FÓRMULA USADA: {}\n".format(formula(f,a,h)))
print("x_i\tValor exacto\t\tAproximación\t\tError")
print("-\t------------\t\t------------\t\t-----")

for i in range(10):
    print("{}\t{}\t{}\t{}".format(i+1, valores_exactos[i], aproximaciones[i], errores[i]))

FÓRMULA USADA: (-2*f(a) + f(a - h) + f(a + h))/h**2

x_i	Valor exacto		Aproximación		Error
-	------------		------------		-----
1	0.676265098285376	0.6762652615677212	1.63282345311266E-7
2	0.283220364176106	0.2832203809255418	1.67494356717590E-8
3	0.168340319928121	0.1683403230146041	3.08648318014804E-9
4	0.114907312895053	0.11490731566254908	2.76749589911418E-9
5	0.0846224302869937	0.08462243039630835	1.09314654550552E-10
6	0.0654364313639429	0.06543643227274742	9.08804489996307E-10
7	0.0523721743690358	0.05237217326481414	1.10422163079882E-9
8	0.0430109449028751	0.04301094591596666	1.01309156202989E-9
9	0.0360420057237485	0.036042004936120975	7.87627567333526E-10
10	0.0306970066620211	0.03069700671431974	5.22986827455885E-11


4.- Divida el intervalo $[1,2]$ en 100 partes iguales y aplique las fórmulas del rectángulo, Simpson y trapecio compuestas para aproximar la integral en dicho intervalo de $f_1$. Compare dichos resultados.

Comenzamos empleando las fórmulas compuestas del rectángulo, que al tratarse de una partición uniforme, usaremos la siguiente fórmula:

$$\int_a^b f(x)\,dx \simeq h \sum_{i=0}^{n-1} f(\psi_i)$$
con $\psi_i = a+ i h, a+(i+1) h$ o bien $a+ (i+1/2)h$ en los casos del rectángulo izquierda, derecha o punto medio respectivamente.

In [30]:
val_exacto = sp.integrate(f1(z),(z,1,2))
val_exacto

Integral(z**z, (z, 1, 2))

In [31]:
# Como da un valor numérico, habrá que usar quad

import scipy.integrate as spi

val_exacto, error = spi.quad(f1, 1, 2)
val_exacto

2.050446234534731

In [32]:
# Comenzamos con el rectángulo izquierda

def formrectangizda(f,a,b,nx):
    """fórmula compuesta de los rectangulos a izquierda"""
    h = (b-a)/nx
    return h*sum([f(a+i*h) for i in range(0,nx)])

In [33]:
aproximaciones = []

val1 = formrectangizda(f1,1,2,100)
aproximaciones.append(val1)
val1

2.0354943390855573

In [34]:
# Fórmula rectángulo derecha

def formrectangdcha(f,a,b,nx):
    """fórmula compuesta de los rectangulos a la derecha"""
    h = (b-a)/nx
    return h*sum([f(a+(i+1)*h) for i in range(0,nx)])

In [35]:
val2 = formrectangdcha(f1,1,2,100)
aproximaciones.append(val2)
val2

2.065494339085557

In [36]:
# Fórmula rectángulo punto medio

def formrectangptomedio(f,a,b,nx):
    """fórmula compuesta de los rectangulos de punto medio"""
    h = (b-a)/nx
    return h*sum([f(a+(i+1/2)*h) for i in range(0,nx)])

In [37]:
val3 = formrectangptomedio(f1,1,2,100)
aproximaciones.append(val3)
val3

2.050422182392515

Por último, nos falta aplicar las fórmulas compuestas de Simpson y los trapecios para particiones uniformes, que son respectivamente:

$$\frac{h}{3}\,\left(f(a)+2\sum_{i=1}^{n-1}f(x_{2i})
+4\sum_{i=1}^{n}f(x_{2i-1}) + f(b) \right)
$$

$$\frac{h}{2}\left( f(a)+2\sum_{i=1}^{n-1}f(a+i\,h)+f(b)\right)$$ 

In [38]:
# Fórmula compuesta de Simpson

def formsimpson(f,a,b,nx):
    h = (b-a)/nx
    m = int(nx/2) #Redondeamos
    E = f(a)+f(b)
    I = sum([f(a+(2*i-1)*h) for i in range(1,m+1)])
    P = sum([f(a+2*i*h) for i in range(1,m)])

    return h/3*(E+2*P+4*I)

In [39]:
val4 = formsimpson(f1,1,2,100)
aproximaciones.append(val4)
val4

2.050446235955426

In [40]:
# Fórmula compuesta de los trapecios

def formtrapecios(f,a,b,nx):
    """fórmula compuesta de los trapecios"""
    h = (b-a)/nx
    return h/2*(f(a)+2*sum([f(a+i*h) for i in range(1,nx)])+f(b))

In [41]:
val5 = formtrapecios(f1,1,2,100)
aproximaciones.append(val5)
val5

2.0504943390855574

In [42]:
# Ahora calculamos los errores asociados a las 5 aproximaciones

errores = []

errores = []
for i in range(5):
    errores.append(abs(val_exacto - aproximaciones[i]))

In [43]:
# Y para hacer la comparación final, mostramos en formato de tabla las 5 aproximaciones con sus respectivos errores

print("Valor \"exacto\": ´{}\n".format(val_exacto)) #Entre comillas porque no es exacto

print("Fórmula\t\tAproximación\t\tError")
print("-------\t\t------------\t\t-----")
print("{}\t{}\t{}".format('formrectangizda', aproximaciones[0], errores[0]))
print("{}\t{}\t{}".format('formrectangdcha', aproximaciones[1], errores[1]))
print("{}\t{}\t{}".format('formrectangptomedio', aproximaciones[2], errores[2]))
print("{}\t{}\t{}".format('formsimpson', aproximaciones[3], errores[3]))
print("{}\t{}\t{}".format('formtrapecios', aproximaciones[4], errores[4]))

Valor "exacto": ´2.050446234534731

Fórmula		Aproximación		Error
-------		------------		-----
formrectangizda	2.0354943390855573	0.014951895449173858
formrectangdcha	2.065494339085557	0.015048104550825947
formrectangptomedio	2.050422182392515	2.4052142216124395e-05
formsimpson	2.050446235955426	1.4206946730155323e-09
formtrapecios	2.0504943390855574	4.8104550826266745e-05


5.- Repita el ejercicio 4 para $f_2$. 

In [44]:
val_exacto = sp.integrate(f2(z),(z,1,2))
val_exacto = val_exacto.evalf()
val_exacto

3.77658111776791

In [45]:
aproximaciones = []

val1 = formrectangizda(f2,1,2,100)
aproximaciones.append(val1)
val1

val2= formrectangdcha(f2,1,2,100)
aproximaciones.append(val2)
val2

val3=formrectangptomedio(f2,1,2,100)
aproximaciones.append(val3)
val3

val4=formsimpson(f2,1,2,100)
aproximaciones.append(val4)
val4

val5=formtrapecios(f2,1,2,100)
aproximaciones.append(val5)
val5

3.77658469845732

In [46]:
errores = []

errores = []
for i in range(5):
    errores.append(abs(val_exacto - aproximaciones[i]))

In [47]:
print("Valor \"exacto\": ´{}\n".format(val_exacto)) #Entre comillas porque no es exacto

print("Fórmula\t\tAproximación\t\tError")
print("-------\t\t------------\t\t-----")
print("{}\t{}\t{}".format('formrectangizda', aproximaciones[0], errores[0]))
print("{}\t{}\t{}".format('formrectangdcha', aproximaciones[1], errores[1]))
print("{}\t{}\t{}".format('formrectangptomedio', aproximaciones[2], errores[2]))
print("{}\t{}\t{}".format('formsimpson', aproximaciones[3], errores[3]))
print("{}\t{}\t{}".format('formtrapecios', aproximaciones[4], errores[4]))

Valor "exacto": ´3.77658111776791

Fórmula		Aproximación		Error
-------		------------		-----
formrectangizda	3.778523202782093	0.00194208501418380
formrectangdcha	3.774646194132547	0.00193492363536230
formrectangptomedio	3.7765793274267083	0.00000179034120106891
formsimpson	3.776581117805272	3.73625574923153E-11
formtrapecios	3.77658469845732	0.00000358068941075373


6.- Sea $f_3(x)=x^{15} e^x$ en $[0,2]$. Vamos a dividir el intervalo en $10\times 2^n$ subintervalos, es decir, $10,\,20,\,40,\, 80,\ldots $ y a aplicar la fórmula de Simpson compuesta hasta que la diferencia entre dos aproximaciones consecutivas (por ejemplo, podrían
ser con $20$ y $40$ subintervalos) sea menor que $10^{-2}$, dando en tal caso por buena la última aproximación obtenida. Programe
y calcule dicha aproximación. Compare ambas aproximaciones con
el valor exacto.

In [48]:
def f3(z):
    return (z**15)*(sp.exp(z))

In [49]:
val_exacto = sp.integrate(f3(z),(z,0,2))
val_exacto = val_exacto.evalf()
val_exacto

27062.7024138996

In [50]:
aproximacion0 = formsimpson(f3,0,2,10).evalf()
aproximacion1 = formsimpson(f3,0,2,20).evalf()

if abs(aproximacion0 - aproximacion1) < 10**(-2):
    print("La última aproximación obtenida ha sido: {}".format(aproximacion1))
else:
    n = 2
    while abs(aproximacion0 - aproximacion1) >= 10**(-2) :
        aproximacion0 = aproximacion1
        aproximacion1 = formsimpson(f3,0,2,10*(2**n)).evalf()
        n+=1
    
    print("La última aproximación obtenida ha sido: {}".format(aproximacion1))
    print("Número de subintervalos utilizados: {}".format(10*(2**(n-1))))

La última aproximación obtenida ha sido: 27062.7024808912
Número de subintervalos utilizados: 640


In [51]:
print("El error asociado a la penúltima aproximación es: {}".format(abs(val_exacto-aproximacion0)))
print("El error asociado a la última aproximación es: {}".format(abs(val_exacto-aproximacion1)))

El error asociado a la penúltima aproximación es: 0.00107168317117612
El error asociado a la última aproximación es: 0.0000669916116748936


7.- Calcule las fórmulas gaussianas con $2$ y $3$ nodos,en el intervalo $[-1,1]$, siendo la función peso el valor absoluto de la variable. Aplíquelas para aproximar la función $x\; e^x$ en $[-1,1]$ y compare los resultados con el valor exacto (organizando los cálculos de forma adecuada).

In [59]:
x = sp.Symbol('x')
def w(x):
    """función peso"""
    return abs(x)

In [60]:
# Definimos una función que nos calcule los nodos de Gauss

def nodosgauss(w,a,b,n):
    
    x = sp.Symbol('x')
    grexact = 2*n-1
    
    # Generamos una tupla de n nodos
    
    p = sp.symbols('p0:'+ str(n))
    nodos = list(p)
    
    # Generamos una tupla de n coeficientes
    
    c = sp.symbols('c0:'+ str(n))
    coefs = list(c)
    
    # nodos = [p(i) for i in range(n)]
    # coefs = [c(i) for i in range(n)]
    
    incognitas = nodos + coefs
    
    ecs = [np.dot([(z**i).subs({z:nodos[j]}) for j in range(n)],coefs)-sp.integrate(w(x)*x**i,(x,a,b)) for i in range(grexact+1)]
    
    solsGauss = sp.solve(ecs,incognitas)
    
    for i in range(n):
        nodos[i] = solsGauss[0][i]
        coefs[i] = solsGauss[0][n+i]
    
    return nodos,coefs

In [61]:
def formgaussiana(f,nodos,coefs,n):
    
    resultado = 0
    
    for i in range(n):
        resultado += coefs[i]*f(nodos[i])
    
    return resultado

In [62]:
# Ya que tenemos la funciones listas, vamos a definir f y hallar primero el valor exacto de su integral

def f4(x):
    return x*sp.exp(x)

In [63]:
val_exacto = sp.integrate(w(x)*f4(x),(x,-1,1))
val_exacto = val_exacto.evalf()
val_exacto

0.557679034316257

In [64]:
nodos, coefs = nodosgauss(w,-1,1,2)
aproximacion2 = formgaussiana(f4,nodos,coefs,2)

nodos, coefs = nodosgauss(w,-1,1,3)
aproximacion3 = formgaussiana(f4,nodos,coefs,3)

In [66]:
# Finalmente comparamos las dos aproximaciones con el valor exacto de la integral y calculamos sus respectivos errores

print("Valor exacto: {}\n".format(val_exacto))

print("n\tAproximación\t\tError")
print("-\t------------\t\t-----")
print("{}\t{}\t{}".format(2, aproximacion2.evalf(), abs(aproximacion2 - val_exacto).evalf()))
print("{}\t{}\t{}".format(3, aproximacion3.evalf(), abs(aproximacion3 - val_exacto).evalf()))

Valor exacto: 0.557679034316257

n	Aproximación		Error
-	------------		-----
2	0.542720820636303	0.0149582136799534
3	0.557437075708894	0.000241958607363272


8.- Programar las técnicas de integración de Romberg y adaptativa, para después aplicarlas a la aproximación de la siguiente integral $$\int_a^b p(x)\, dx$$
siendo  $\;a=\displaystyle\min_{0\leq i\leq 7}{d_i}$, $\;b=\displaystyle\max_{0\leq i\leq 7}{d_i}$ y 
$$p(x)=d_0 + d_1 x + d_2 x^2 + d_3 x^3+ d_4 x^4 + d_5 x^5 + d_6 x^6 + d_7 x^7 $$
(siendo $d_0, d_1, \ldots, d_7$ los dígitos de su DNI, pasaporte o tarjeta de residente).

Como mi DNI es 76067985,  $\;a=\displaystyle\min_{0\leq i\leq 7}{d_i} = 0$, $\;b=\displaystyle\max_{0\leq i\leq 7}{d_i} = 9$

Tenemos que calcular $$\int_0^9 7 + 6x + 6x^3 + 7x^4 + 9x^5 + 8x^6 + 5x^7\, dx$$, así como aproximarla por las técnicas de integradión de Romberg y adaptativa.

In [67]:
def p(x):
    return 7 + 6*x + 6*x**3 + 7*x**4 + 9*x**5 + 8*x**6 + 5*x**7

In [68]:
val_exacto = sp.integrate(p(x),(x,0,9))
val_exacto = val_exacto.evalf()
val_exacto

33260428.5107143

Comenzaremos programando el algoritmo de integración de Romberg:

###### Algoritmo de integración de Romberg

La integración de Romberg consiste en hallar una sucesión, $\{R(N,N)\}_{N=0}^{\infty}$.

$$
\begin{matrix}
R(0,0) \\
R(1,0) & R(1,1) \\
R(2,0) & R(2,1) & R(2,2) \\
\vdots & \vdots & \vdots & \ddots \\
R(N,0) & R(N,1) & R(N,2) & \cdots & R(N,N)
\end{matrix}
$$

Como vimos en teoría, el cálculo para los $R(i,j)$ es:

$$
R(j,0) = T_{2j}, \quad R(j,k) = \frac{4^k R(j,k-1)- R(j-1,k-1)}{4^k-1} \qquad \text{con} \quad j= 0,1,...,N; k = 1,...,j
$$

con $T_n$ la función `formtrapecios` del ejercicio 4.

In [69]:
def romberg(f,a,b,tol=10**(-5)): # Nuestro criterio de parada es que dos aproximaciones consecutivas se diferencien menos
                                 # de 10**(-5)
    n = 0
    r0 = [formtrapecios(f,a,b,2**n)]
    condicion = True

    while condicion:
        n += 1
        
        # Se calcula R(n,0)
        r1 = [formtrapecios(f,a,b,2**n)]
        
        # Se calculan todos los R(n,i) con 1 <= i <= n (es decir, la fila r1)
        for i in range(1,n):
            r1.append((4**i*r1[i-1]-r0[i-1])/(4**i-1))
        
        # Se comparan los últimos elementos de las filas, que son justo los R de la diagonal
        condicion = (abs(r1[n-1] - r0[n-2]) >= tol)
        
        # La fila r1 se copia en r0 para empezar a generar la fila r1 en la siguiente iteración
        r0 = r1

    return r1[n-1]

In [70]:
aproximacion = romberg(p,0,9)
aproximacion

33260428.51071429

In [71]:
# Finalmente calculamos el error asociado a esta aproximación

error_romberg = abs(val_exacto - aproximacion)
error_romberg

3.72529029846191e-9

##### Algoritmo de integración adaptativa

Se define la fórmula de Simpson adaptativa: $\int_{a}^{b} f(x)dx \approx S(a,b)$ siendo $S(a,b) = \frac{b-a}{6} (f(a) + 4f(m) + f(b))$.

El algoritmo se basa en hallar $S(a,b)$, $S(a,m)$ y $S(m,b)$ siendo $m = \frac{a+b}{2}$ hasta que $|S(a,b) - S(a,m) - S(m,b)| < 10\epsilon$. De esta forma, se puede afirmar que $\int_{a}^{b} f(x)dx \approx S(a,m) + S(m,b)$ con un error $< \epsilon$. Si esto no ocurre, se aplica el algoritmo en $[a,m]$ y $[m,b]$ con error $\frac{\epsilon}{2}$ en cada uno de los dos subintervalos.

In [72]:
def formsimpsonadaptativa(f,a,b):
    return ((b-a)/6)*(f(a)+4*f((a+b)/2)+f(b))

In [73]:
def integracionadaptativa(f,a,b, error = 10**(-4)):
    
    m = (a+b)/2
    resultado = 0
    if(abs(formsimpsonadaptativa(f,a,b)-formsimpsonadaptativa(f,a,m)-formsimpsonadaptativa(f,m,b))<10*error):
        resultado = formsimpsonadaptativa(f,a,m) + formsimpsonadaptativa(f,m,b)
    else:
        resultado = integracionadaptativa(f,a,m,error/2) + integracionadaptativa(f,m,b,error/2)
    
    return resultado

In [74]:
aproximacion = integracionadaptativa(p,0,9)
aproximacion

33260428.510740392

In [75]:
# Finalmente calculamos el error asociado a la aproximación obtenida por integración adaptativa

error_adaptativa = abs(val_exacto - aproximacion)
error_adaptativa

2.61068344116211e-5

In [76]:
error_romberg < error_adaptativa

True

Concluimos que el error de aproximación de integración de Romberg es menor que el de integración adaptativa.